In [1]:

import torch as t
import torch.nn as nn 
import torch.nn.functional as F 
from torch import Tensor
from jaxtyping import Int, Float
from dataclasses import dataclass
import einops

In [ ]:
class StridedAttention(nn.Module):
    def __init__(self, num_heads=2, d_model=4, num_horizontal_strides=2):
        super().__init__()
        self.num_heads = num_heads
        self.num_horizontal_strides = num_horizontal_strides
        
        # d_head = d_model // num_heads
        self.W_Q = nn.Parameter(t.randn(d_model, d_model))
        self.W_K = nn.Parameter(t.randn(d_model, d_model))
        self.W_V = nn.Parameter(t.randn(d_model, d_model))

    def forward(self, embed: Float[Tensor, "b s d_m"]) -> Float[Tensor, "b s d_m"]:
        b, s, d_m = embed.shape
        
        Q = embed @ self.W_Q # b s_q d_m
        K = embed @ self.W_K # b s_k d_m
        V = embed @ self.W_V # b s_v d_
        
        stride_length = s // self.num_horizontal_strides
        """
        q = s_q d_m
        kt = d_m s_k -> s_q s_k
        s_q * d_m subset_k -> s_q subset_k
        """
        strided_K = K[:, ::stride_length]

        # b 1 s_q d_m
        Q = Q[:, None, ...]
        # TODO: Think of a non memory moving way to do this.
        # b 2 s_q d_head
        Q = Q.repeat(1,2,1,1)

        # b 1 s_k d_head
        K = K[:, None, ...]
        V = V[:, None, ...]
        # stride_width = seq_len // num_heads
        print(V.shape)

        # horizontal stride shape b 1 s_k//stride d_head
        K = K[:, :, range(0, s, self.horizontal_stride), :]
        print(K.shape)

        # range 1..... seq_Q
        # seq_Q, s_k//stride

        

        # K_t = K.transpose(1, 2) # b d_head s_k
        # Q_K_t = Q @ K_t # GOAL: b s_q s_k//stride
        
        # sf(qk/scale)v


B=4
S=128
D=256
input = t.randn((B, S, D))
print(input.shape)
attn = StridedAttention(2, D, D//2)        
attn(input)


torch.Size([4, 128, 256])
torch.Size([4, 1, 128, 128])
torch.Size([4, 1, 64, 128])


In [3]:
import numpy as np

In [4]:
vals =np.random.randint(1, 9, size=(6,6))

In [33]:
n = 6

# (row + col) % stride
"""
    
    F F T F F T
    T F F T F F
    F T F F T F
    
"""
stride_length = 3
rows = np.arange(n)[:, None]
cols = np.arange(n-1, -1, -1)[None, :]
mask = (rows + cols) % stride_length == 0

rows_to_pull = np.where(mask, np.arange(n)[:, None], -1)

In [39]:
vals

array([[3, 7, 7, 1, 6, 5],
       [5, 3, 8, 8, 3, 4],
       [7, 7, 7, 4, 5, 8],
       [3, 7, 4, 1, 5, 7],
       [7, 3, 5, 8, 8, 3],
       [1, 5, 4, 1, 3, 6]])

In [40]:
mask

array([[False, False,  True, False, False,  True],
       [ True, False, False,  True, False, False],
       [False,  True, False, False,  True, False],
       [False, False,  True, False, False,  True],
       [ True, False, False,  True, False, False],
       [False,  True, False, False,  True, False]])

In [42]:
self_attn_mask = np.tril(n)

TypeError: tri() missing 1 required positional argument: 'N'

In [37]:
vals[mask].reshape((n, n//stride_length))

array([[7, 5],
       [5, 8],
       [7, 5],
       [4, 7],
       [7, 8],
       [5, 3]])

In [29]:
test = t.arange(0, 3)[:, None] + t.arange(0, 24, 8)[None, :]
print(test.shape)
print(test)


torch.Size([3, 3])
tensor([[ 0,  8, 16],
        [ 1,  9, 17],
        [ 2, 10, 18]])


In [ ]:
toks = t.tensor([
    [1,2,3,4,5,6],
    [7,8,9,10,11,12],
])

In [ ]:
seq_len, d_model = toks.shape
num_heads = 2
head_dim = d_model // num_heads

viewed = toks.view(num_heads, seq_len, head_dim)

In [8]:
viewed[1, :, :]

tensor([[ 7,  8,  9],
        [10, 11, 12]])